# Handcrafted Feature
In this section, we introduce handcrafted features for a point cloud data. The handcrafted feature is data encoded of shape property and can be extracted on a point-by-point from raw data such as coordinates or colors. A common use case with the handcrafted feature is the registration task. An example of a registration pipeline using Handcrafted fetature is shown below. Registration is the relative coordinate estimation task between 2 point clouds and uses the handcrafted features to find correspondence points between 2 point clouds:
1. Pre-processing select the point to apply the handcrafted feature and then estimate normals for selected points.
2. Feature extraction computes handcrafted features of the points from normal and coordinates.
3. Post-processing estimates transformation matrix between two point clouds with correspondences between points based on distances in the feature space.

![processing](img/registration_with_handcrafted_feature.png)

Section 4 of [Rusu, 2010] explain background of handcrafted features: 

> In their native representation, points as defined in the concept of 3D mapping systems are simply represented using their Cartesian coordinates $x, y, z$, with respect to a given origin. Assuming that the origin of the coordinate system does not change over time, there could be two points $p_1$ and $p_2$ , acquired at $t_1$ and $t_2$ , having the same coordinates. Comparing these points however is an ill-posed problem, because even though they are equal with respect to some distance measure (e.g. Euclidean metric), they could be sampled on completely different surfaces, and thus represent totally different information when taken together with the other surrounding points in their vicinity. That is because there are no guarantees that the world has not changed between $t_1$ and $t_2$. Some acquisition devices might provide extra information for a sampled point, such as an intensity or surface remission value, or even a color, however that does not solve the problem completely and the comparison remains ambiguous.

> Applications which need to compare points for various reasons require better characteristics and metrics to be able to distinguish between geometric surfaces. The concept of a 3D point as a singular entity with Cartesian coordinates therefore disappears, and a new concept, that of local descriptor takes its place. The literature is abundant of different naming schemes describing the same conceptualization, such as shape descriptors or geometric features but for the remaining of this document they will be referred to as point feature representations.

In this section, we introduce a usage example and methods of the handcrafted feature.
- Registration task solution with the handcrafted feature
- PFH (Point Feature Histograms)
- FPFH (Fast Point Feature Histograms)
- PPF (Point Pair Feature)


In [1]:
%load_ext autoreload
%autoreload 2

## Registration task solution with the handcrafted feature
This subsection will demonstrate an usage example of the Point Feature Histogram (PFH) with registration task. PFH is a handcrafted feature that supplies a histogram between relationships of neighbor features.

An example of Registration is shown below.


In [2]:
# for registration
from tutlibs.sampling import voxel_grid_sampling
from tutlibs.registration import feature_ransac
from tutlibs.feature import PointFeatureHistograms as pfh
from tutlibs.normal_estimation import normal_estimation, normal_orientation

# for description
import numpy as np
from tutlibs.transformation import TransformationMatrix as tm
from tutlibs.transformation import Transformation as tr
from tutlibs.io import Points as io
from tutlibs.visualization import JupyterVisualizer as jv
from tutlibs.utils import single_color


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [6]:
# processing for target point cloud
target_xyz, _, _ = io.read("../data/redwood_indoor_1.ply")
target_ds_xyz = voxel_grid_sampling(target_xyz, 0.05)
target_ds_normal = normal_estimation(target_ds_xyz, k=15)
target_ds_normal = normal_orientation(target_ds_xyz, target_ds_normal)

# processing for source point cloud
source_xyz, _, _ = io.read("../data/redwood_indoor_2.ply")
source_xyz = tr.rotation(source_xyz, "y", np.pi / 3)
source_ds_xyz = voxel_grid_sampling(source_xyz, 0.05)
source_ds_normal = normal_estimation(source_ds_xyz, k=15)
source_ds_normal = normal_orientation(source_ds_xyz, source_ds_normal)

# extract Handcrafted feature
source_ds_pfh = pfh.compute(source_ds_xyz, source_ds_normal, 0.25)
target_ds_pfh = pfh.compute(target_ds_xyz, target_ds_normal, 0.25)

# feature matching
trans_mat = feature_ransac(
    source_ds_xyz, target_ds_xyz, source_ds_pfh, target_ds_pfh, 3, 10000, 0.05
)
trans_source_xyz = tm.transformation(source_ds_xyz, trans_mat)

# visualize results
obj_source_points = jv.point(
    source_ds_xyz, single_color("#ff0000`", len(source_ds_xyz))
)
obj_trans_source_points = jv.point(
    trans_source_xyz, single_color("#00ff00", len(trans_source_xyz))
)
obj_target_points = jv.point(target_ds_xyz, single_color("#ffff00", len(target_ds_xyz)))
jv.display([obj_trans_source_points, obj_target_points, obj_source_points])


Output()

In the above implementation, we estimate the position of source points relative to target points. The red point cloud is the initial position of source points, yellow is the transformed source point cloud, and green is the target point cloud.
The method with PFH is more robust than the simple method (ex: ICP) since PFH includes relative angle features between normals that do not depend on object position and direction.
Because this output shows that the source point cloud nearly overlaps with the target point cloud, PFH features are good in this registration example.

There are handcrafted features other than PFH. In the next subsection, we introduce handcrafted features and its detail.

## PFH (Point Feature Histograms)
PFH [Rusu et al, 2008] is a histogram feature of relative angles and distances between neighbor points.

This subsection use the following code:

In [2]:
from tutlibs.normal_estimation import normal_estimation
from tutlibs.feature import PointFeatureHistograms as pfh
from tutlibs.feature import pair_feature

from tutlibs.io import Points as io
import inspect

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
xyz, _, _ = io.read('../data/bunny_pc.ply')
normals = normal_estimation(xyz, 15)
pfhs = pfh.compute(xyz, normals)
print('PFH shape: {}'.format(pfhs.shape))

PFH shape: (4093, 125)


`pfh.compute` outputs a 125 bin histogram by point from coordinates and a normal.
The following output shows a content of `pfh.compute`.

In [4]:
print(inspect.getsource(pfh.compute))

    @staticmethod
    def compute(
        xyz: np.ndarray,
        normals: np.ndarray,
        radius=0.03,
        div: int = 5,
        normalization: bool = True,
    ) -> np.ndarray:
        """Compute Point Feature Histograms.
        Note: does not include the distance feature (reason -> https://pcl.readthedocs.io/projects/tutorials/en/latest/pfh_estimation.html#pfh-estimation)

        Args:
            xyz: xyz coords (N, 3)
            normals: normals (N, 3)
            radius: radius for nearest neighbor search
            div: number of subdivisions of value range for each feature.
            normalization: normalize each point histgram.

        Return:
            point feature histograms: (N, div**3)
        """
        knn_idxs, _ = radius_nearest_neighbors(xyz, xyz, r=radius)

        # define PFH list
        num_pf = len(knn_idxs)  # num_pf = N = xyz.shape[0]
        pfh_list = np.zeros((num_pf, div ** 3))

        for i in range(num_pf):
            # Get point s

In above implementation, 

1. get neighbors for each point by `radius_nearest_neighbors` function. In the figure below, we use an any point as the query point (red point) and neighbors (blue points) within a fixed distance (inside the dotted line) from the query point.
2. calculate PFH for each point. To calculate the PFH of a point, we use the query point and the neighbor points obtained on 1.
   1. create all combinations of two points with the query point and its neighbors. The figure shows two point combinations as edges.
   2. use the `pair_feature` function to obtain the feature between points for each combination.
   3. obtain PFH that split the features to bins of histogram.

![nn](img/pfh_nn.png)  
[by Rusu et al, 2009]

Next, we show `pair_feature` to compute feature between points.

In [5]:
print(inspect.getsource(pair_feature))

def pair_feature(xyz:np.ndarray, normals:np.ndarray, pair_idxs:np.ndarray):
    """Compute pair features.

    Args:
        xyz: xyz coords (M, 3)
        normals: normals (M, 3)
        pair_idxs: point pair indices (L, 2)
    Return:
        phi: cosine feature, value range is -1~1 (L)
        alpha: cosine feature, value range is -1~1 (L)
        theta: angle feature, value range is -pi/2~pi/2 (L)
        dists: distance feature (L)
    """
    # Get xyz and normal of points
    p1 = xyz[pair_idxs[:, 0]]
    n1 = normals[pair_idxs[:, 0]]
    p2 = xyz[pair_idxs[:, 1]]
    n2 = normals[pair_idxs[:, 1]]

    # Get vectors (pp) and distances (dists) between pt and ps
    pp = p2 - p1
    dists = np.linalg.norm(pp, ord=2, axis=1)

    # Get a mask to decide the source and target points.
    p1pp_angle = dot(n1, pp) / dists
    p2pp_angle = dot(n2, pp) / dists
    mask = np.arccos(np.fabs(p1pp_angle)) > np.arccos(np.fabs(p2pp_angle))

    # Decide source and target points.
    phi = p1pp

In above implementation, we input coordinates, normals and $L$ pair indices to `pair_feature` and then `pair_feature` outputs features of each pair (pair feature). Pair features are 3 relative angle between normals and relative postion ($\phi$, $\alpha$ and $\theta$) and one relative distance ($d$). Note that the implementation does not use $d$ when compute PFH.

Let the two points be $P_S$ and $P_T$, respectively. $P_S$ have the smaller angle between the self normal and the relative direction of two points than $P_T$:

$$
\begin{aligned}
&\text{if } \left\langle n_{i}, p_{j}-p_{i}\right\rangle \leq\left\langle n_{j}, p_{i}-p_{j}\right\rangle \\
&\text{then } p_{s}=p_{i}, p_{t}=p_{j} \\
&\text{else } p_{s}=p_{j}, p_{t}=p_{i}  
\end{aligned}
$$

The pair features are then as follows.

![pf](img/pfh_pf.png)  
[by Rusu et al, 2010]

$$

\begin{aligned}
&\alpha =\left\langle v, n_{t}\right\rangle \\
&d =\left\|p_{t}-p_{s}\right\| \\
&\phi =\left\langle u, p_{t}-p_{s}\right\rangle / f_{2} \\
&\theta=\operatorname{atan}\left(\left\langle w, n_{t}\right\rangle,\left\langle u, n_{t}\right\rangle\right)
\end{aligned}

$$



## FPFH (Fast Point Feature Histograms)
FPFH [Rusu et al, 2008] is a histogram feature of relative angles and distances between neighbor points.

This subsection use the following code:

In [ ]:
from tutlibs.normal_estimation import normal_estimation
from tutlibs.feature import PointFeatureHistograms as pfh
from tutlibs.feature import pair_feature

from tutlibs.io import Points as io
import inspect

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
xyz, _, _ = io.read('../data/bunny_pc.ply')
normals = normal_estimation(xyz, 15)
pfhs = pfh.compute(xyz, normals)
print('PFH shape: {}'.format(pfhs.shape))

PFH shape: (4093, 125)


In [ ]:
print(inspect.getsource(pfh.compute))

    @staticmethod
    def compute(
        xyz: np.ndarray,
        normals: np.ndarray,
        radius=0.03,
        div: int = 5,
        normalization: bool = True,
    ) -> np.ndarray:
        """Compute Point Feature Histograms.
        Note: does not include the distance feature (reason -> https://pcl.readthedocs.io/projects/tutorials/en/latest/pfh_estimation.html#pfh-estimation)

        Args:
            xyz: xyz coords (N, 3)
            normals: normals (N, 3)
            radius: radius for nearest neighbor search
            div: number of subdivisions of value range for each feature.
            normalization: normalize each point histgram.

        Return:
            point feature histograms: (N, div**3)
        """
        knn_idxs, _ = radius_nearest_neighbors(xyz, xyz, r=radius)

        # define PFH list
        num_pf = len(knn_idxs)  # num_pf = N = xyz.shape[0]
        pfh_list = np.zeros((num_pf, div ** 3))

        for i in range(num_pf):
            # Get point s

## PPF (Point Pair Feature)
PFF [Drost et al, 2010] is features that have the relative distance and normal angle between 2 points.

This subsection use the following code:

In [6]:
from tutlibs.normal_estimation import normal_estimation
from tutlibs.feature import PairPointFeature as ppf

from tutlibs.io import Points as io

In [ ]:
xyz, rgb, _ = io.read('../data/bunny_pc.ply')
normals = normal_estimation(xyz)
ppfs = ppf.compute(xyz, normals)
print('PPF shape: {}'.format(ppfs.shape))

## References
- [Rusu, R. B., N. Blodow, Z. C. Marton, and M. Beetz. 2008. “Aligning Point Cloud Views Using Persistent Feature Histograms.” In 2008 IEEE/RSJ International Conference on Intelligent Robots and Systems. IEEE. https://doi.org/10.1109/iros.2008.4650967.](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.391.5915&rep=rep1&type=pdf)
- [Rusu, Radu Bogdan, Nico Blodow, and Michael Beetz. 2009. “Fast Point Feature Histograms (FPFH) for 3D Registration.” In 2009 IEEE International Conference on Robotics and Automation, 3212–17.](https://www.cvl.iis.u-tokyo.ac.jp/class2016/2016w/papers/6.3DdataProcessing/Rusu_FPFH_ICRA2009.pdf)
- [Drost, Bertram, Markus Ulrich, Nassir Navab, and Slobodan Ilic. 2010. “Model Globally, Match Locally: Efficient and Robust 3D Object Recognition.” In 2010 IEEE Computer Society Conference on Computer Vision and Pattern Recognition. IEEE. https://doi.org/10.1109/cvpr.2010.5540108.](http://campar.in.tum.de/pub/drost2010CVPR/drost2010CVPR.pdf)
- [Rusu, R.B. Semantic 3D Object Maps for Everyday Manipulation in Human Living Environments. Künstl Intell 24, 345–348 (2010). https://doi.org/10.1007/s13218-010-0059-6](https://link.springer.com/article/10.1007/s13218-010-0059-6#citeas)
- [Rusu, Radu Bogdan, Zoltan Csaba Marton, Nico Blodow, and Michael Beetz. 2008. “Learning Informative Point Classes for the Acquisition of Object Model Maps.” In 2008 10th International Conference on Control, Automation, Robotics and Vision. IEEE. https://doi.org/10.1109/icarcv.2008.4795593.](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.584.2647&rep=rep1&type=pdf)